In [6]:
import pandas as pd
import numpy as np
from numpy.random import default_rng

# Introduction
This notebook explores the classic percolation model for bushfire modelling and considers the Poisson distribution later.

In [49]:
def create_lattice(p, N, mode):
    rng = default_rng(12345)
    if mode == 'bernoulli':
        x = rng.uniform(low=0, high=1, size=(N, N))
        y = x < p
    elif mode == 'poisson':
        y = rng.poisson(lam=p, size=(N, N))
    return y.astype(int)

# returns false if index is outside of lattice
def in_lattice(index, lattice):
    m, n = lattice.shape
    i, j = index
    if not 0 <= i <= m-1 or not 0 <= j <= n-1:
        return False
    else:
        return True

def get_neighbours(node, lattice):
    # get neighbours based on reach, i.e. node value
    # same for bernoulli and poisson modes, reach = L_inf distance
    nbs = []
    i, j = node
    reach = lattice[node]
#     k + l <= reach
    for k in range(reach + 1):
        for l in range(reach + 1 - k):
            for signk in [-1, 1]:
                for signl in [-1, 1]:
                    nb = (i + signk*k, j + signl* l)
                    if in_lattice(nb, lattice):
                        nbs.append(nb)
            
    # remove duplicates from k=0 or l=0
    nbs = list(set(nbs))
    # remove original node, case k=l=0
    if node in nbs:
        nbs.remove(node)
    return nbs
    
    

## Testing

In [50]:
x = create_lattice(.5, 5, 'bernoulli')
x

array([[1, 1, 0, 0, 1],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 1],
       [0, 0, 1, 0, 1],
       [1, 1, 1, 1, 1]])

In [21]:
y = create_lattice(.5, 5, 'poisson')
y

array([[0, 0, 1, 0, 0],
       [0, 0, 2, 2, 0],
       [2, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 2, 1, 1]])

In [32]:
index = (0, 0)
print(in_lattice(index, x))
print(in_lattice((0, 4), x))


True
True


In [42]:
get_neighbours((0, 1), x)

[(0, 2), (1, 1), (0, 0)]

In [51]:
def simulate(p, N, mode):
    matrices = []
    lattice = create_lattice(p, N, mode)
    front = []
    archive = []
    
    # add top row to front
    I, J = np.where(lattice[:1] > 0)
    for index in zip(I, J):
        front.append(index)
        
    # spread fire
    while front:
        new_front = []
        mat = lattice.copy()
#         mat = np.copy(lattice)
        matrices.append(mat)
        
        for node in front:
            nbs = get_neighbours(node, lattice)
            for nb in nbs:
                if not nb in front and not nb in archive:
                    new_front.append(nb)
                    lattice[nb] = -1
        front = new_front
    return matrices

In [53]:
mats = simulate(.5, 5, 'bernoulli')